In [1]:
import sys
import numpy as np
np.random.seed(0)
import keras.backend as K
from keras import initializers
from keras.engine import InputSpec
from keras.layers import Dense, Lambda, Wrapper


class ConcreteDropout(Wrapper):
    """This wrapper allows to learn the dropout probability for any given input Dense layer.
    ```python
        # as the first layer in a model
        model = Sequential()
        model.add(ConcreteDropout(Dense(8), input_shape=(16)))
        # now model.output_shape == (None, 8)
        # subsequent layers: no need for input_shape
        model.add(ConcreteDropout(Dense(32)))
        # now model.output_shape == (None, 32)
    ```
    `ConcreteDropout` can be used with arbitrary layers which have 2D
    kernels, not just `Dense`. However, Conv2D layers require different
    weighing of the regulariser (use SpatialConcreteDropout instead).
    # Arguments
        layer: a layer instance.
        weight_regularizer:
            A positive number which satisfies
                $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$ (inverse observation noise),
            and N the number of instances in the dataset.
            Note that kernel_regularizer is not needed.
        dropout_regularizer:
            A positive number which satisfies
                $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and N the number of
            instances in the dataset.
            Note the relation between dropout_regularizer and weight_regularizer:
                $weight_regularizer / dropout_regularizer = l**2 / 2$
            with prior lengthscale l. Note also that the factor of two should be
            ignored for cross-entropy loss, and used only for the eculedian loss.
    """

    def __init__(self, layer, weight_regularizer=1e-6, dropout_regularizer=1e-5,
                 init_min=0.1, init_max=0.1, is_mc_dropout=True, **kwargs):
        assert 'kernel_regularizer' not in kwargs
        super(ConcreteDropout, self).__init__(layer, **kwargs)
        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        self.is_mc_dropout = is_mc_dropout
        self.supports_masking = True
        self.p_logit = None
        self.p = None
        self.init_min = np.log(init_min) - np.log(1. - init_min)
        self.init_max = np.log(init_max) - np.log(1. - init_max)

    def build(self, input_shape=None):
        self.input_spec = InputSpec(shape=input_shape)
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(ConcreteDropout, self).build()  # this is very weird.. we must call super before we add new losses

        # initialise p
        self.p_logit = self.layer.add_weight(name='p_logit',
                                            shape=(1,),
                                            initializer=initializers.RandomUniform(self.init_min, self.init_max),
                                            trainable=True)
        self.p = K.sigmoid(self.p_logit[0])

        # initialise regulariser / prior KL term
        assert len(input_shape) == 2, 'this wrapper only supports Dense layers'
        input_dim = np.prod(input_shape[-1])  # we drop only last dim
        weight = self.layer.kernel
        kernel_regularizer = self.weight_regularizer * K.sum(K.square(weight)) / (1. - self.p)
        dropout_regularizer = self.p * K.log(self.p)
        dropout_regularizer += (1. - self.p) * K.log(1. - self.p)
        dropout_regularizer *= self.dropout_regularizer * input_dim
        regularizer = K.sum(kernel_regularizer + dropout_regularizer)
        self.layer.add_loss(regularizer)

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def concrete_dropout(self, x):
        '''
        Concrete dropout - used at training time (gradients can be propagated)
        :param x: input
        :return:  approx. dropped out input
        '''
        eps = K.cast_to_floatx(K.epsilon())
        temp = 0.1

        unif_noise = K.random_uniform(shape=K.shape(x))
        drop_prob = (
            K.log(self.p + eps)
            - K.log(1. - self.p + eps)
            + K.log(unif_noise + eps)
            - K.log(1. - unif_noise + eps)
        )
        drop_prob = K.sigmoid(drop_prob / temp)
        random_tensor = 1. - drop_prob

        retain_prob = 1. - self.p
        x *= random_tensor
        x /= retain_prob
        return x

    def call(self, inputs, training=None):
        if self.is_mc_dropout:
            return self.layer.call(self.concrete_dropout(inputs))
        else:
            def relaxed_dropped_inputs():
                return self.layer.call(self.concrete_dropout(inputs))
            return K.in_train_phase(relaxed_dropped_inputs,
                                    self.layer.call(inputs),
                                    training=training)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from functools import partial
from UQ_in_ML.general_utils import *

def set_problem(problem, n_data):
    if problem == 'cubic':
        var_n = 9.
        f = partial(f_cubic, var_n=var_n)
        xn = np.random.uniform(low=-4.5, high=4.5, size=(n_data,1))
        bounds = (-6, 6)
    elif problem == 'sin_cos':
        var_n = 0.1**2
        f = partial(f_sin_cos, var_n=var_n)
        bounds = (0, 10)
        xn = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_data,1))
    yn = f(xn, noisy=True)
    x_plot = np.linspace(bounds[0], bounds[1], 100).reshape((-1,1))
    fig, ax = plt.subplots(ncols=1, figsize=(6, 4))
    y_plot = f(x_plot, noisy=False)
    ax.plot(x_plot, y_plot, color='green', label='true function')
    ax.plot(xn, yn, color='blue', marker='.', linestyle='none', label='data')
    plt.legend()
    plt.show()
    return f, var_n, bounds, xn, yn, x_plot, y_plot

def plot_regressor(reg, x_plot, y_plot, xn, yn, title):
    fig, ax = plt.subplots(ncols=1, figsize=(6, 4))
    ax.plot(x_plot, y_plot, color='green', label='true', alpha=0.5)
    plot_UQ(reg, X=x_plot, ax=ax, plot_one_posterior=True)
    ax.plot(xn, yn, linestyle='none',marker='.',label='data')
    ax.legend()
    ax.set_title(title)
    plt.show()
    
# Choose the problem, and the network architecture
problem = 'sin_cos' #'cubic' or 'sin_cos'
n_data = 20
f, var_n, bounds, xn, yn, x_plot, y_plot = set_problem(problem, n_data)

<Figure size 600x400 with 1 Axes>

In [3]:
from keras.layers import Input, Dense, Lambda, merge
from keras.models import Model
from keras import backend as K

precision = 1 / var_n
nb_features = 10
batch_size = n_data

def fit_model(l, nb_epoch, X, Y):
    if K.backend() == 'tensorflow':
        K.clear_session()
    N = X.shape[0]
    wd = l**2. / N
    dd = 2. / N
    inp = Input(shape=(1,))
    x = inp
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    x = ConcreteDropout(Dense(nb_features, activation='relu'), weight_regularizer=wd, dropout_regularizer=dd)(x)
    mean = ConcreteDropout(Dense(1), weight_regularizer=wd, dropout_regularizer=dd)(x)
    model = Model(inp, mean)
    
    def homoscedastic_loss(true, pred):
        return K.sum(precision * (true - pred)**2., -1)
    
    model.compile(optimizer='adam', loss=homoscedastic_loss)
    assert len(model.layers[1].trainable_weights) == 3  # kernel, bias, and dropout prob
    assert len(model.losses) == 4  # a loss for each Concrete Dropout layer
    hist = model.fit(X, Y, nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)
    loss = hist.history['loss'][-1]
    return model, -0.5 * loss  # return ELBO up to const.

In [ ]:
l = 1e-4

nb_epoch = 100
model, ELBO = fit_model(l, nb_epoch, xn, yn)
print(ELBO)
print(K.eval(K.sigmoid(model.layers[1].get_weights()[2])))
print(K.eval(K.sigmoid(model.layers[2].get_weights()[2])))

nb_epoch = 500
model, ELBO = fit_model(l, nb_epoch, xn, yn)
print(ELBO)
print(K.eval(K.sigmoid(model.layers[1].get_weights()[2])))
print(K.eval(K.sigmoid(model.layers[2].get_weights()[2])))

nb_epoch = 2000
model, ELBO = fit_model(l, nb_epoch, xn, yn)
print(ELBO)
print(K.eval(K.sigmoid(model.layers[1].get_weights()[2])))
print(K.eval(K.sigmoid(model.layers[2].get_weights()[2])))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


-8.798266410827637
[0.09782438]
[0.0983275]
-11.224080085754395
[0.09803983]
[0.09413876]


In [ ]:
l = 1e-2

nb_epoch = 100
model, ELBO = fit_model(l, nb_epoch, xn, yn)

print(K.eval(K.sigmoid(model.layers[1].get_weights()[2])))
print(K.eval(K.sigmoid(model.layers[2].get_weights()[2])))

nb_epoch = 20000
model, ELBO = fit_model(l, nb_epoch, xn, yn)
print(K.eval(K.sigmoid(model.layers[1].get_weights()[2])))
print(K.eval(K.sigmoid(model.layers[2].get_weights()[2])))